In [9]:
import tensorflow as tf
import json
import os
import logging  # To log errors or missing files

## Images rename to image_id

In [20]:
os.getcwd()

'D:\\Projects\\DL\\MonumentDetection\\DL\\keras_cv'

In [27]:
os.chdir('coco-nyaptola')

In [31]:
os.getcwd()

'D:\\Projects\\DL\\MonumentDetection\\DL\\keras_cv\\coco-nyaptola'

In [35]:
count = 0
image_dir = 'images'
for file in os.listdir(image_dir):
    old_filename = os.path.join(image_dir, file)
    new_filename = os.path.join(image_dir, str(count) + '.png')
    os.rename(old_filename, new_filename)
    count += 1

## Keras_CV Compatible Annotation file

In [36]:
def load_and_convert_annotations(annotations_file, output_file):
  with open(annotations_file, 'r') as f:
    annotations = json.load(f)

  xywh_annotations = convert_coco_to_xywh(annotations)

  with open(output_file, 'w') as f:
    json.dump(xywh_annotations, f, indent=2) 

In [37]:
def convert_coco_to_xywh(annotations):
  #Converts bounding boxes in COCO format to xywh format.
  xywh_annotations = []
  for ann in annotations["annotations"]:
    image_id = ann["image_id"]
    bbox = ann["bbox"]  # Assuming bbox is in [xmin, ymin, width, height] format
    x_min, y_min, width, height = bbox

    # Convert to xywh format
    x_center = x_min + width / 2
    y_center = y_min + height / 2

    xywh_annotations.append({
      "image_id": image_id,
      "bbox": [x_center, y_center, width, height],
      "category_id": ann["category_id"],  # Assuming category_id exists
    })
  return xywh_annotations

In [40]:
annotation_file = "result.json"
annotation_file = os.path.abspath(annotation_file)
output_file = "annotations.json"

In [41]:
converted_annotations = load_and_convert_annotations(annotation_file,output_file)

## Conversion of annotations to yolo format

In [2]:
!pip install xmltodict


DEPRECATION: Loading egg at c:\users\acer\anaconda3\envs\object_detection\lib\site-packages\avro_python3-1.10.2-py3.9.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [4]:
import xmltodict
import os

# Function to convert XML to YOLO format
def convert_annotation(xml_file):
    with open(xml_file, 'r') as file:
        annotation = xmltodict.parse(file.read())['annotation']
    
    yolo_annotations = []
    
    # Check if 'object' is a list or a single item
    objects = annotation.get('object', [])
    
    if isinstance(objects, dict):
        # If it's a dictionary, convert it to a list for consistent processing
        objects = [objects]

    for obj in objects:
        class_name = obj['name']
        bbox = obj['bndbox']
        
        xmin = int(bbox['xmin'])
        ymin = int(bbox['ymin'])
        xmax = int(bbox['xmax'])
        ymax = int(bbox['ymax'])
        
        # Calculate YOLO format
        image_width = int(annotation['size']['width'])
        image_height = int(annotation['size']['height'])

        x_center = (xmin + xmax) / 2 / image_width
        y_center = (ymin + ymax) / 2 / image_height
        width = (xmax - xmin) / image_width
        height = (ymax - ymin) / image_height
        
        class_id = 0  # Adjust according to your class mapping
        yolo_annotations.append(f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}")

    return yolo_annotations


In [5]:
# Loop over all XML annotation files
for annotation_file in os.listdir(annotations_path):
    if annotation_file.endswith('.xml'):
        xml_path = os.path.join(annotations_path, annotation_file)
        
        # Convert XML to YOLO format
        yolo_annotations = convert_annotation(xml_path)
        
        # Create corresponding text file for YOLO
        txt_filename = annotation_file.replace('.xml', '.txt')
        txt_path = os.path.join(output_path, txt_filename)
        
        with open(txt_path, 'w') as txt_file:
            txt_file.write('\n'.join(yolo_annotations))

In [6]:
import os
import shutil
import random

# Set paths
images_path = "dataset/images/"
yolo_labels_path = "dataset/yolo_labels/"

# Create directories for training and validation
os.makedirs("dataset/train/images", exist_ok=True)
os.makedirs("dataset/train/labels", exist_ok=True)
os.makedirs("dataset/val/images", exist_ok=True)
os.makedirs("dataset/val/labels", exist_ok=True)

# List all images
images = os.listdir(images_path)

# Shuffle images and split into training and validation
random.shuffle(images)
train_ratio = 0.8
num_train = int(len(images) * train_ratio)

train_images = images[:num_train]
val_images = images[num_train:]

# Move images and corresponding labels to respective directories
for image in train_images:
    base_name = os.path.splitext(image)[0]
    label_file = f"{base_name}.txt"
    
    shutil.copy(os.path.join(images_path, image), "dataset/train/images/")
    shutil.copy(os.path.join(yolo_labels_path, label_file), "dataset/train/labels/")

for image in val_images:
    base_name = os.path.splitext(image)[0]
    label_file = f"{base_name}.txt"
    
    shutil.copy(os.path.join(images_path, image), "dataset/val/images/")
    shutil.copy(os.path.join(yolo_labels_path, label_file), "dataset/val/labels/")


## Another Attempt

In [11]:
import tensorflow as tf
import os

def load_sample_with_shape(image_path, label_path):
    # Load the image
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [256, 256])  # Resize or reshape as necessary

    # Load the annotations
    label = tf.io.read_file(label_path)
    label = tf.strings.split(label, sep='\n')
    label = tf.strings.to_number(label, tf.float32)
    label = tf.reshape(label, [-1, 4])  # Assuming each label has 4 values (e.g., bounding box coordinates)

    return image, label

def filter_empty_annotations(image, annotations):
    # Filter out samples where annotations are empty
    return tf.size(annotations) > 0

def pad_annotations(image, annotations, max_num_annotations=100):
    num_annotations = tf.shape(annotations)[0]
    pad_size = max_num_annotations - num_annotations

    # Ensure pad_size is non-negative
    pad_size = tf.maximum(pad_size, 0)

    # Create padding tensor dynamically
    padding = tf.stack([[0, pad_size], [0, 0]], axis=0)
    padded_annotations = tf.pad(annotations, padding, constant_values=-1)
    return image, padded_annotations

def data_loader(images_dir, labels_dir, batch_size):
    image_files_pattern = os.path.join(images_dir, "*.jpg")
    label_files_pattern = os.path.join(labels_dir, "*.txt")

    image_files = tf.data.Dataset.list_files(image_files_pattern, shuffle=False)
    label_files = tf.data.Dataset.list_files(label_files_pattern, shuffle=False)

    # Print the matched files for debugging
    tf.print("Matched image files:")
    for file in image_files.take(5):
        tf.print(file)
    tf.print("Matched label files:")
    for file in label_files.take(5):
        tf.print(file)

    # Check if the datasets are empty
    if tf.data.experimental.cardinality(image_files) == 0:
        raise FileNotFoundError(f"No image files found in {images_dir} with pattern {image_files_pattern}")
    if tf.data.experimental.cardinality(label_files) == 0:
        raise FileNotFoundError(f"No label files found in {labels_dir} with pattern {label_files_pattern}")

    dataset = tf.data.Dataset.zip((image_files, label_files))

    dataset = dataset.map(load_sample_with_shape, num_parallel_calls=tf.data.AUTOTUNE)

    dataset = dataset.filter(lambda image, annotations: tf.py_function(
        func=filter_empty_annotations,
        inp=[image, annotations],
        Tout=tf.bool)
    )

    dataset = dataset.map(lambda image, annotations: pad_annotations(image, annotations), num_parallel_calls=tf.data.AUTOTUNE)

    dataset = dataset.batch(batch_size, drop_remainder=False).prefetch(tf.data.AUTOTUNE)

    return dataset

# Usage
train_dataset = data_loader("dataset/train/images", "dataset/train/labels", batch_size=3)

# Check the validation dataset paths and files
try:
    val_dataset = data_loader("dataset/val/images", "dataset/val/labels", batch_size=3)
except FileNotFoundError as e:
    print(e)


Matched image files:
dataset\train\images\55c04d26-Nyathpola.jpg
Matched label files:
dataset\train\labels\115e44a7-image_29.txt
dataset\train\labels\14a405f1-image_10.txt
dataset\train\labels\159954d7-image_199.txt
dataset\train\labels\174afc0c-image_86.txt
dataset\train\labels\1c2822e4-image_480.txt


InvalidArgumentError: Expected 'tf.Tensor(False, shape=(), dtype=bool)' to be true. Summarized data: b'No files matched pattern: dataset/val/images\\*.jpg'

In [7]:
os.getcwd()

'D:\\Projects\\DL\\MonumentDetection\\DL'